#### To Do
- Don't upload if already exists in bucket folder
- Make unzip_blob a subflow?
- Convert to parquet before uploading to blob again?
- Delete zip blob, once unzip complete
- Unzip all folders
- Check bucket size
- Use pyspark or dbt to get into BQ?

In [85]:
from io import BytesIO
from pathlib import Path
from zipfile import ZipFile, is_zipfile
import pandas as pd
from prefect import flow, task
from google.cloud import storage
import os

In [86]:
# @task()
def read_gcs_bucket(bucket) -> list:
    """Get list of zip folders in GCS bucket"""
    lsblob = list(
        bucket.list_blobs(prefix="zip")
    )  # get list of blobs in zip folder of bucket
    lsblob = [l.name for l in lsblob]  # only return the filename from the blobs
    print(lsblob)

    return lsblob

In [87]:
# @task()
def unzip_blob(source_path: str, bucket) -> list:
    """Unzip folder"""
    blob = bucket.blob(source_path)

    zipbytes = BytesIO(blob.download_as_string())

    if is_zipfile(zipbytes):
        with ZipFile(zipbytes, "r") as myzip:
            for contentfilename in myzip.namelist():
                folder = contentfilename.split("_")[0]
                contentfile = myzip.read(contentfilename)

                tmp_dir = "../tmp"
                Path(tmp_dir).mkdir(parents=True, exist_ok=True)

                trunc_fn = contentfilename.split("_")[1]
                output_file = f"../tmp/{trunc_fn}"
                outfile = open(output_file, "wb")
                outfile.write(contentfile)
                outfile.close()

                if folder == "catch":
                    blob = bucket.blob(f"catch/{trunc_fn}")
                elif folder == "trip":
                    blob = bucket.blob(f"trip/{trunc_fn}")
                elif folder == "size":
                    blob = bucket.blob(f"size/{trunc_fn}")

                with open(output_file, "rb") as mycsv:
                    blob.upload_from_file(mycsv)

                os.remove(f"../tmp/{trunc_fn}")  # delete/remove outfile

    # tmp_dir = '../tmp'
    # Path(tmp_dir).mkdir(parents=True, exist_ok=True)

    # with open('../tmp/archive.zip', 'wb') as file_obj:
    #     blob.download_to_file(file_obj)

    # zip_archive = ZipFile('../tmp/archive.zip', 'r')

    # destination_blob = bucket.blob(destination_path)
    # zip_archive.extractall(destination_path)

    # return zip_archive

In [88]:
# @task()
def convert_to_parquet():
    """Upload local parquet file to GCS"""

In [89]:
# @flow(log_prints=True)
def process_gcs_blob() -> None:
    """Process blob in gcs bucket"""
    storage_client = storage.Client.from_service_account_json("../creds.json")
    bucket_name = "de_project_bucket"  # parameterize this
    bucket = storage_client.get_bucket(bucket_name)

    lsblob = read_gcs_bucket(bucket)

    source = lsblob[0]
    lsfiles = unzip_blob(source, bucket)

    # for blob in lsblob:
    #     lsfiles = unzip_blob(blob, bucket)

In [90]:
# if __name__ == '__main__':
process_gcs_blob()


['zip/ps_1981_1989_csv.zip', 'zip/ps_1990_1994_csv.zip', 'zip/ps_1995_1999_csv.zip', 'zip/ps_2000_2004_csv.zip', 'zip/ps_2005_2009_csv.zip', 'zip/ps_2010_2014_csv.zip', 'zip/ps_2015_2016_csv.zip', 'zip/ps_2017_csv.zip', 'zip/ps_2018_csv.zip', 'zip/ps_2019_csv.zip', 'zip/ps_2020_csv.zip', 'zip/ps_2021_csv.zip']
